In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/w266')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/w266')

In [1]:
import sys
import os
import json
import re

import pandas as pd
import numpy as np

In [8]:
os.chdir(r'/Users/thebobs/Downloads/stocknet-dataset/tweet/raw')

# preprocessing tweets

In [5]:
def clean(tweet):
    # tweet = re.sub(r'http\S+|www\S+|@\S+|#\S+|[^A-Za-z\s]', '', tweet)
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)
    tweet=re.sub(r"#(\w+)",'', tweet)
    tweet = re.sub('([_]+)', "", tweet)
    if not re.search(r'[.,!?;:"]$',tweet[-1]):
        tweet+=". "
    return tweet

In [156]:
def extract(ticker,date):
    data=[]
    with open(f'/Users/thebobs/Downloads/stocknet-dataset/tweet/raw/{ticker}/{date}') as f:
        for line in f:
            j=json.loads(line)
            if not 'retweeted_status' in j:
                text=clean(j['text'])
                if len(re.findall(r'\b\w+\b', text))>=10:
                    data.append((date,text,ticker))
    return data

In [157]:
def extract_ticker_all(ticker):
    if not ticker.startswith("."):
        data=[]
        for date in os.listdir(f'/Users/thebobs/Downloads/stocknet-dataset/tweet/raw/{ticker}'):
            data+=extract(ticker,date)
        return pd.DataFrame(data,columns=['date','tweets','ticker'])

In [158]:
extract_ticker_all("AAPL")

,date,tweets,ticker
0,2014-10-14,$AAPL - Retail Sales Seen Down 0.1%--Data Week...,AAPL
1,2014-10-14,"Skeptics ask, ""How many more devices can Apple...",AAPL
2,2014-10-14,"$AAPL News: ""14 Undervalued Investment Ideas A...",AAPL
3,2014-10-14,$S SPRINT will let U lease iPhone 6 or 6 Plus ...,AAPL
4,2014-10-14,Sell $AAPL!!! Make 100% a year using our 'Top ...,AAPL
...,...,...,...
10168,2015-09-13,"RT Sourcingstocks: had great calls on $TWTR, ...",AAPL
10169,2015-09-13,richytee Bourne2Prosper mmnnyy1111 IGPJay zing...,AAPL
10170,2015-09-13,DoktaRufas iSonnySood TraderFTW Ziggityzola Se...,AAPL
10171,2015-09-13,AntonioJesusDR AbdHakeemSabani JHarfordTyrer H...,AAPL


In [25]:
# for ticker in os.listdir()[2:]:
#     if not ticker.startswith('.'):
#         data=extract_ticker_all(ticker)
#         data.to_csv(f'../clean/{ticker}.csv',index=False)


In [159]:
tweet_all=pd.concat([extract_ticker_all(i) for i in os.listdir(f'/Users/thebobs/Downloads/stocknet-dataset/tweet/raw/')])

In [162]:
tweet_all.tail()

,date,tweets,ticker
20,2016-01-20,This Stock Has A 9.93% Yield And Sells For Les...,CODI
21,2014-09-07,Stock Contest!! Pick $GOOGL and WIN a FREE Tab...,CODI
22,2015-05-18,CODI Compass Diversified Holdings 52Wk Low\nCl...,CODI
23,2014-08-09,$CODI COMPASS DIVERSIFIED HOLDINGS Files SEC f...,CODI
24,2015-11-11,Compass Diversified Holdings Stock Rating Reaf...,CODI


In [167]:
tweet_all.dropna().sort_values(['ticker','date']).to_csv('tweets_all.csv',index=False)

In [164]:
tweet_all.dropna().shape

(62677, 3)

# preprocess price

In [32]:
os.chdir(r'/Users/thebobs/Downloads/stocknet-dataset/price/raw')

In [33]:
data_all=[]
for i in os.listdir():
    data_all.append(pd.read_csv(i,index_col=0,parse_dates=True)['Adj Close'].rename(i[:-4]))
p=pd.concat(data_all,axis=1)

In [168]:
p.shape

(1258, 88)

In [ ]:
p.to_csv('price.csv')

In [36]:
r_now=np.log(p).diff()
r_next=np.log(p).diff(2).shift(-2)

In [37]:
def zscore(data):
    return (data-data.mean())/data.std()

In [38]:
r_now=zscore(r_now)
r_next=zscore(r_next)

In [42]:
os.chdir('..')

In [43]:
os.getcwd()

'/Users/thebobs/Downloads/stocknet-dataset'

In [67]:
t=pd.read_csv(f'tweet/clean/{ticker}.csv',index_col=0,parse_dates=True)

In [70]:
t.columns

Index(['tweets'], dtype='object')

In [94]:
def match_text_label(ticker,r_now,r_next):
    try:
        tweets=extract_ticker_all(ticker)
        tweets['date']=pd.to_datetime(tweets['date'])
    except FileNotFoundError:
        return
    merged=tweets.merge(r_now[ticker].rename('r_now').reset_index(),left_on='date', right_on='Date').drop('Date',axis=1)
    final=merged.merge(r_next[ticker].rename('r_next').reset_index(),left_on='date', right_on='Date').drop('Date',axis=1)
    final['ticker']=ticker
    return final

In [95]:
match_text_label("D",r_now,r_next)

,date,tweets,r_now,r_next,ticker
0,2014-10-14,La deuda de Yacyreta es de casi U$D 18 mil mil...,-2.701183,-0.282104,D
1,2016-03-03,Simmons First Trust Co. NA Purchases Shares of...,1.733848,1.199413,D
2,2015-05-07,;8B;X&amp;u4/bC=WNz&lt;J-QdB{2Y&gt;Bik1;SIg$=F...,0.267350,0.013681,D
3,2014-08-20,""": Jackelin Farias deberia explicar donde esta...",0.039994,-0.214766,D
4,2014-08-20,$B$D$P6eO:JFJ[Ev$NCf?H$OIaDL$NJ[(B$BEv(B .,0.039994,-0.214766,D
...,...,...,...,...,...
711,2014-07-14,obstart();\n foreach($theactivities as $theact...,-0.925558,0.844790,D
712,2014-07-14,Seeking the next stock like $CNCA $NOV $D $WTW...,-0.925558,0.844790,D
713,2015-09-14,Gana U$D 9.843 al año con Clixsense trabajando...,0.474522,0.612263,D
714,2015-11-18,I'm ridin round with my system bumpin L$D.,0.219291,0.482142,D


In [96]:
pairs=pd.concat([match_text_label(t,r_now,r_next) for t in r_next.columns])

In [107]:
os.getcwd()

'/Users/thebobs/Downloads/stocknet-dataset'

In [108]:
pairs.dropna().to_csv('clean_data.csv',index=False,encoding="UTF-8")

In [106]:
?pairs.to_csv

Signature:
pairs.to_csv(
    path_or_buf: 'FilePath | WriteBuffer[bytes] | WriteBuffer[str] | None' = None,
    sep: 'str' = ',',
    na_rep: 'str' = '',
    float_format: 'str | Callable | None' = None,
    columns: 'Sequence[Hashable] | None' = None,
    header: 'bool_t | list[str]' = True,
    index: 'bool_t' = True,
    index_label: 'IndexLabel | None' = None,
    mode: 'str' = 'w',
    encoding: 'str | None' = None,
    compression: 'CompressionOptions' = 'infer',
    quoting: 'int | None' = None,
    quotechar: 'str' = '"',
    lineterminator: 'str | None' = None,
    chunksize: 'int | None' = None,
    date_format: 'str | None' = None,
    doublequote: 'bool_t' = True,
    escapechar: 'str | None' = None,
    decimal: 'str' = '.',
    errors: 'str' = 'strict',
    storage_options: 'StorageOptions' = None,
) -> 'str | None'
Docstring:
Write object to a comma-separated values (csv) file.

Parameters
----------
path_or_buf : str, path object, file-like object, or None, default None